In [2]:
import numpy as np

In [22]:
# 4차원 데이터 형성 => 5개의 데이터 1채널 28x28형태
x = np.random.rand(5, 1, 7, 7)
print(x.shape)


(5, 1, 7, 7)


In [4]:
y = np.zeros((10, 1, 3, 3, 5, 5))
print(y.shape)

(10, 1, 3, 3, 5, 5)


In [5]:
test = np.array([[1, 2, 3], [4, 5, 6]])
print(test.shape)

(2, 3)


In [31]:
# 합성곱 계층에서 합성곱 연산을 빠르게 하기 위해 데이터를 2차원 행렬 형태로 나타내주는 함수 im2col
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    
    # padding
    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]
    
    # reshape할때 데이터의 원하는 정렬을 위해 transpose를 취해준다
    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    
    return col

In [34]:
col = im2col(x, 3, 3, stride=1, pad=0)
print(col.shape)

(125, 9)


In [35]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, input_data):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = input_data.shape
        
        out_h = (H - FH + 2*pad) / self.stride + 1
        out_w = (W - FW + 2*pad) / self.stride + 1
        
        input_col = im2col(input_data, FH, FW, self.stride, self.pad) # (out_h*out_w*1) x (C*FH*FW) 형태의 2차원 배열
        f_col = self.W.reshape(FN, -1).T # (C*FH*FW) x FN 형태의 2차원 배열
        
        output_col = np.dot(input_col, f_col) + self.b # (out_h*out_w*1) x FN 형태의 2차원 배열
        out = output_col.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out
    

In [ ]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
        # 역전파에 사용
        self.db = None
        self.out = None
        
        
        
    def forward(self, input_data):
        N, C, H, W = input_data.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        # 전개
        col = im2col(input_data, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)
        # 최대값을 뽑아낸 뒤 4차원 데이터 형태로 변형
        out = np.max(col, axis=1)
        self.out = out
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out
    
    def backward(self, dout):
        
        